In [1]:
from acc2sf_offline import get_cadence
from media_player import Player
import numpy as np
import time
import pandas as pd
#from HR_2_optimal_HR import *

In [ ]:
# This module adapts music tempo to HR by the following rules:
# current HR & cadnece above/below optimal  -> speed up/down tempo
# every stabilization_period, a decision for changing the tempo will be taken
# this modules ignores warm up period

# initiate audio player
player = Player(["/mnt/c/Users/Owner/OneDrive/Desktop/Technion/semester6/project_a/BiofeedbackAssistedMusic/recordings/running_track_sunshine_jaunt.mp3"])

# module initialization
stabilization_period = 15 # in sec. divisuble by 10
delta = 10
#warm_up_time = 100 # in sec
runner_age = 26
runner_max_HR = 220 - runner_age # TODO: find more accurate formula
low_HR, high_HR = 95.95, 134.33
optimalHR = np.mean([low_HR, high_HR])
 
# Start of Warming Up period
# time.sleep(warm_up_time)
# optimal_HR = HR_to_optimal_HR(low_HR, high_HR)
# End of Warming Up period

# averaging cadence and HR in spans of 15 sec
cadence = get_cadence("../recordings/walking_metronome_100bpm/ACC.csv") # assuming cadence is measured after warm up period
print(cadence)
cadence = cadence[:len(cadence)-len(cadence)%stabilization_period]
cadence = cadence.reshape(-1, stabilization_period)
cadence = np.mean(cadence, axis=1)

df = pd.read_csv("../recordings/walking_metronome_100bpm/HR.csv", skiprows=2, header=None) #  the signal is computed in spans of 10 sec
df = np.concatenate(df.to_numpy())
df = df[:len(df)-len(df)%stabilization_period]
df = df.reshape(-1, stabilization_period)
HR = np.mean(df, axis=1)

for i, currentHR in enumerate(HR):
    HR_diff = optimalHR - currentHR
    cadence_diff = optimalHR - (cadence[i]*60) # beat per second units

    # TODO: improve the following if conditions
    if(HR_diff > delta and cadence_diff > delta):
        player.up()
        print('tempo up')
    elif(HR_diff < -delta or cadence_diff < -delta):
        player.down(
        print('tempo down')
        )
    else:
        print('tempo the same')
        pass

    # stabilization period
    # time.sleep(stabilization_period)